<a href="https://colab.research.google.com/github/Jeevesh8/Cross-Lingual-Voice-Cloning/blob/master/clvc_infer_gh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##from google.colab import drive
##drive.mount('/content/drive')

In [ ]:
!pip install Unidecode
!pip install tensorflow==1.15.2

In [ ]:
##%cd '/content/drive/My Drive'

In [ ]:
##!git clone https://github.com/Jeevesh8/Cross-Lingual-Voice-Cloning Cross-Lingual-Voice-Cloning-gpu

In [ ]:
##%cd Cross-Lingual-Voice-Cloning-gpu

/content/drive/My Drive/projects/dragoman/Cross-Lingual-Voice-Cloning-gpu-1


In [ ]:
!git submodule init
!git submodule update

In [ ]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import sys
sys.path.append('waveglow/')
import numpy as np
import torch

from hparams import create_hparams
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence

In [ ]:
from model import Tacotron2

In [ ]:
base_dir = '/content/Drive/My Drive'

In [ ]:
%cd ./waveglow
waveglow_path = base_dir+'<waveglow-weights-path>'                              # To get universal vocoder waveglow weights--> https://drive.google.com/open?id=1okuUstGoBe_qZ4qUEF8CcwEugHP7GM_b
new_model_path = base_dir+'<waveglow-new-weights-path>'
!python convert_model.py '<waveglow-wts-old-path>'  '<waveglow-wts-new-path>'
%cd ..

In [ ]:
waveglow = torch.load(new_model_path)['model']
waveglow.cuda().eval() #.half()
for k in waveglow.convinv:
    k.float()
#denoiser = Denoiser(waveglow)

In [ ]:
hparams = create_hparams()
hparams.sampling_rate = 22050
hparams.max_decoder_steps = 1000
hparams.n_speakers = 2
hparams.dim_yo = 2
hparams.lang_embedding_dim = 4
#set other hparams you want to change

In [ ]:
print(hparams.dim_yo)

In [ ]:
checkpoint_path = '<path-to-checkout-of model>'
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval() #.half()

In [ ]:
text = "Could there be a talking plot"
sequence = np.array(text_to_sequence(text, ['basic_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()

In [ ]:
print(sequence.shape)

In [ ]:
speaker = torch.tensor([13], device='cuda:0')
lang = torch.tensor([0], device='cuda:0')
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence, speaker, lang)
print(mel_outputs.shape)

In [ ]:
import IPython.display as ipd
with torch.no_grad():
    audio = waveglow.infer(mel_outputs, sigma=0.1)                                    #PUT MEL IN MEL_OUTPUTS_POSTNET
print(audio.shape)
ipd.Audio(audio[0].data.cpu().numpy(), rate=22050)